In [1]:
import mysql.connector as mariadb
import pandas as pd
import psycopg2

financial_conn_string = {
    'host': 'relational.fit.cvut.cz',
    'port': '3306',
    'user': 'guest',
    'password': 'relational',
    'database': 'financial'
}

Firstly I import the necessary library to connect with the MariaDB Database. I then set up the database connection parameters and store this in the 'financial_conn_string' variable.

In [2]:
def fetch_data_from_mariadb(conn_params, query):
    with mariadb.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames

financial_query_transactions = "SELECT * FROM trans;"
financial_data_transactions, transactions_colnames = fetch_data_from_mariadb(financial_conn_string, financial_query_transactions)

Then I define a function to fetch the data from MariaDB. This function takes two arguments:
- conn_params = the database connection
- query = the sql query to execute.
    
I used a 'with' statement to create a connection to the MariaDB database using the 'mariadb.connect()' method. 
I then create a cursor using 'with conn.cursor() as cur' to interact with the database and execute queries.
The query is executed using 'cur.execute(query)' and the result is fetched using 'data = cur.fetchall()' and stored in 'data'. The column names are retrieved from the cursors description using a list comprehension 'colnames = [desc[0] for desc in cur.description]' and stored in 'colnames'. I then return the data and colnames.

I define a variable called 'financial_query_transactions' and put a SQL SELECT query to fetch all records from the 'trans' table. Finally I use the 'fetch_data_from_mariadb()' function to fetch data from the 'trans' table and store this data in the columns 'financial_data_transactions, transactions_colnames'

In [15]:
transactions_df = pd.DataFrame(financial_data_transactions, columns=transactions_colnames)
avg_transaction_amount = transactions_df.groupby('account_id')['amount'].mean().reset_index()
avg_transaction_amount.columns = ['AccountID', 'AverageTransactionAmount']

In this next part of the code, I use the pandas library to manipulate some of the data I fetched.

First I load the fetched data into the dataframe and store this in a variable called 'transactions_df'. The columns parameter is set to 'transactions_colnames', which contained the column names fetched earlier.

To calculate the average transaction amount I group the data in the dataframe based on the 'account_id' column and then calculates the average(mean) of the 'amount' column within each group. This will give the average transaction amount for each unique account ID.

I reset the index of this grouped Dataframe. This converts the group labels (account IDs) back into regular columns.

Lastly I rename the columns to 'AccountID', 'AverageTransactionAmount' for better readability.

In [16]:
etl_bites_conn_string = """
                host='localhost' 
                port='5432' 
                dbname='etl_bites' 
                user='ilhaam.ahmed' 
                password='dataproj'
                """

I use psycopg2 to insert the transformed data in bulk into the local PostgreSQL database (etl_bites).
We will need a connection string as well for the destination database (our local PostgreSQL one).

In [22]:
# Create tables in analytical DB
# This could also be done manually via a GUI (e.g. TablePlus) or with a SQL script
def execute_query_postgresql(conn_string, query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_average_transaction_amount_table = '''
CREATE TABLE average_transaction_amount (
    AccountID INTEGER NOT NULL,
    AverageTransactionAmount NUMERIC(15, 2) NOT NULL
);
'''

execute_query_postgresql(etl_bites_conn_string, create_average_transaction_amount_table)

def insert_data_to_postgresql(conn_string, table_name, data, columns):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

table_name = "average_transaction_amount"
columns = ['AccountID', 'AverageTransactionAmount']
insert_data_to_postgresql(etl_bites_conn_string, table_name, avg_transaction_amount, columns)

First I define a function that is used to execute a SQL query on a postgres database. A cursor is created to execute the query and once the query is executed it is commited to the database.

Then I create a variable that stores the SQL query to create a table named 'average_transaction_amount' in the database.

This function execute_query_postgresql() is used to execute the query in the database.

I then create another function to insert the data into a specified table in the postgres database, it takes four parameters. Inside the function a connection is established with the postgres database. Then a cursor is created to execute the insert queries. For each row in the dataframe, an insert query is constructed using the table_name and columns, The changes are commited to the database.

I define the table_name and the list of column names to be used when inserting the data into the average transaction amount table.

Finally, I use the insert_data_to_postgresql() function to insert data from the dataframe into the table.